In [ ]:
# -----------------------------
# 1. Data Loading and Splitting
# -----------------------------
folder_path = os.path.join("stocks", "datasets", "dj30", "raw", "*.csv")
csv_files = glob.glob(folder_path)
data_frames = []

for file in csv_files:
    symbol = os.path.splitext(os.path.basename(file))[0]
    print(f"Processing {symbol} from {file}...")
    df = pd.read_csv(file, parse_dates=['Date'])
    mask = (df['Date'] >= '2010-01-01') & (df['Date'] <= '2018-12-21')
    df = df.loc[mask]
    df = df[['Date', 'Adj Close']].set_index('Date')
    df.rename(columns={'Adj Close': symbol}, inplace=True)
    data_frames.append(df)

merged_df = pd.concat(data_frames, axis=1)
merged_df.sort_index(inplace=True)

# Split data:
train_data = merged_df.loc['2010-01-01':'2016-01-01']
test_data  = merged_df.loc['2016-02-01':'2018-02-01']

# -----------------------------
# 2. Calibration on Training Data (for Simulation)
# -----------------------------
train_log_returns = np.log(train_data / train_data.shift(1)).dropna()
window_size = 120
correlation_matrices = []
for start in range(0, len(train_log_returns) - window_size + 1, window_size):
    window_data = train_log_returns.iloc[start:start+window_size]
    corr_matrix = window_data.corr().values
    correlation_matrices.append(corr_matrix)

n_assets = train_log_returns.shape[1]
features = [mat[np.triu_indices(n_assets, k=1)] for mat in correlation_matrices]
features = np.array(features)

Z = linkage(features, method='ward')
n_clusters = 4
clusters = fcluster(Z, t=n_clusters, criterion='maxclust')

rep_corr_matrices = []
for cluster_id in range(1, n_clusters + 1):
    indices = np.where(clusters == cluster_id)[0]
    avg_corr = np.mean([correlation_matrices[i] for i in indices], axis=0)
    rep_corr_matrices.append(avg_corr)

annual_factor = 252
annual_drift = train_log_returns.mean() * annual_factor
annual_vol = train_log_returns.std() * np.sqrt(annual_factor)
mu = annual_drift.values    
sigmas = annual_vol.values

S0s = train_data.iloc[-1].values

T = 1       # 1 year
dt = 1 / 252
n_steps = int(T / dt)

def simulate_correlated_prices(S0s, mu, sigmas, T, dt, corr_matrix):
    n_assets = len(S0s)
    n_steps = int(T / dt)
    prices = np.zeros((n_steps + 1, n_assets))
    prices[0] = S0s
    L = np.linalg.cholesky(corr_matrix)
    for t in range(1, n_steps + 1):
        Z = np.random.normal(0, 1, size=n_assets)
        correlated_Z = L @ Z
        prices[t] = prices[t-1] * np.exp((mu - 0.5 * sigmas**2) * dt + sigmas * np.sqrt(dt) * correlated_Z)
    return prices

def simulate_multiple_paths(S0s, mu, sigmas, T, dt, corr_matrix, num_paths=1000):
    sims = []
    for _ in range(num_paths):
        sim = simulate_correlated_prices(S0s, mu, sigmas, T, dt, corr_matrix)
        sims.append(sim)
    return np.array(sims)

all_simulations = {}
num_paths = 1000
for idx, corr in enumerate(rep_corr_matrices):
    sims = simulate_multiple_paths(S0s, mu, sigmas, T, dt, corr, num_paths=num_paths)
    all_simulations[idx] = sims

combined_simulations = np.concatenate(list(all_simulations.values()), axis=0)
dummy_dates = pd.date_range(start="2018-01-01", periods=combined_simulations.shape[1], freq="B")
asset_names = [f"Stock{i+1}" for i in range(combined_simulations.shape[2])]
train_episode_df = pd.DataFrame(combined_simulations[np.random.choice(combined_simulations.shape[0])],
                                index=dummy_dates, columns=asset_names)

# -----------------------------
# 3. RL Framework (Environment, Agent, Training)
# -----------------------------
class PortfolioEnv:
    def __init__(self, price_data, window_obs=60, window_state=120):
        self.price_data = price_data.reset_index(drop=True)
        self.window_obs = window_obs
        self.window_state = window_state
        self.n_assets = price_data.shape[1]
        self.reset()
    
    def reset(self):
        self.current_step = max(self.window_obs, self.window_state)
        self.done = False
        self.portfolio_value = 1.0
        self.history = [self.portfolio_value]
        self.weights = np.ones(self.n_assets) / self.n_assets
        return self._get_state()
    
    def step(self, action):
        self.weights = action
        current_prices = self.price_data.iloc[self.current_step].values
        next_prices = self.price_data.iloc[self.current_step+1].values
        asset_returns = (next_prices / current_prices) - 1
        portfolio_return = np.dot(self.weights, asset_returns)
        self.portfolio_value *= (1 + portfolio_return)
        self.history.append(self.portfolio_value)
        self.current_step += 1
        if self.current_step >= len(self.price_data)-1:
            self.done = True
        reward = portfolio_return
        return self._get_state(), reward, self.done, {}
    
    def _get_state(self):
        obs_data = self.price_data.iloc[self.current_step - self.window_obs:self.current_step]
        obs_returns = np.log(obs_data / obs_data.shift(1)).dropna().values.T
        if obs_returns.shape[1] < self.window_obs:
            pad = np.zeros((self.n_assets, self.window_obs - obs_returns.shape[1]))
            obs_returns = np.hstack((obs_returns, pad))
        hist = np.array(self.history)
        if len(hist) < self.window_state+1:
            state_data = np.log(hist[1:] / hist[:-1])
            state_data = np.pad(state_data, (self.window_state - len(state_data), 0), 'constant')
        else:
            window_hist = hist[-(self.window_state+1):]
            state_data = np.log(window_hist[1:] / window_hist[:-1])
        state_data = state_data[np.newaxis, :]
        obs_tensor = torch.tensor(obs_returns, dtype=torch.float32).unsqueeze(0)
        state_tensor = torch.tensor(state_data, dtype=torch.float32).unsqueeze(0)
        return (obs_tensor, state_tensor)
    
    def render(self):
        plt.plot(self.history)
        plt.title("Portfolio Value")
        plt.show()

def generate_actions(n_assets, n_actions=50):
    actions = []
    for _ in range(n_actions):
        w = np.random.rand(n_assets)
        w = w / w.sum()
        actions.append(w)
    return np.array(actions)

actions = generate_actions(train_episode_df.shape[1], n_actions=50)

class RLPortfolioManager(nn.Module):
    def __init__(self, obs_channels, obs_length, state_channels, state_length, num_actions):
        super(RLPortfolioManager, self).__init__()
        self.obs_conv = nn.Sequential(
            nn.Conv1d(in_channels=obs_channels, out_channels=32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv1d(32, 32, kernel_size=3, stride=1),
            nn.ReLU()
        )
        conv_obs_length = obs_length - 4
        self.fc_obs = nn.Linear(32 * conv_obs_length, 128)
        
        self.state_conv = nn.Sequential(
            nn.Conv1d(in_channels=state_channels, out_channels=16, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv1d(16, 16, kernel_size=3, stride=1),
            nn.ReLU()
        )
        conv_state_length = state_length - 4
        self.fc_state = nn.Linear(16 * conv_state_length, 128)
        
        self.fc_combined = nn.Linear(128 + 128, 128)
        self.policy_head = nn.Linear(128, num_actions)
        self.value_head = nn.Linear(128, 1)
        
    def forward(self, observation, state):
        obs_features = self.obs_conv(observation)
        obs_features = obs_features.view(obs_features.size(0), -1)
        obs_features = F.relu(self.fc_obs(obs_features))
        
        state_features = self.state_conv(state)
        state_features = state_features.view(state_features.size(0), -1)
        state_features = F.relu(self.fc_state(state_features))
        
        combined = torch.cat([obs_features, state_features], dim=1)
        combined = F.relu(self.fc_combined(combined))
        policy_logits = self.policy_head(combined)
        value = self.value_head(combined)
        return policy_logits, value

obs_channels = train_episode_df.shape[1]
obs_length = 60
state_channels = 1
state_length = 120
num_actions = actions.shape[0]

policy_net = RLPortfolioManager(obs_channels, obs_length, state_channels, state_length, num_actions)
optimizer = optim.Adam(policy_net.parameters(), lr=1e-4)

def train_agent(policy_net, optimizer, actions, combined_simulations, num_episodes=500, window_obs=60, window_state=120, gamma=0.99):
    for ep in range(num_episodes):
        episode_idx = np.random.choice(combined_simulations.shape[0])
        dummy_dates = pd.date_range(start="2018-01-01", periods=combined_simulations.shape[1], freq="B")
        asset_names = [f"Stock{i+1}" for i in range(combined_simulations.shape[2])]
        train_episode_df = pd.DataFrame(combined_simulations[episode_idx],
                                        index=dummy_dates, columns=asset_names)
        
        env = PortfolioEnv(train_episode_df, window_obs=window_obs, window_state=window_state)
        state = env.reset()
        
        log_probs = []
        values = []
        rewards = []
        done = False
        
        while not done:
            obs, port_state = state
            policy_logits, value = policy_net(obs, port_state)
            probs = F.softmax(policy_logits, dim=1)
            m = torch.distributions.Categorical(probs)
            action_idx = m.sample()
            log_prob = m.log_prob(action_idx)
            value = value.squeeze(1)
            action = actions[action_idx.item()]
            
            next_state, reward, done, _ = env.step(action)
            log_probs.append(log_prob)
            values.append(value)
            rewards.append(reward)
            state = next_state
        
        R = 0
        returns = []
        for r in reversed(rewards):
            R = r + gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns, dtype=torch.float32)
        
        values = torch.stack(values).squeeze(1)
        log_probs = torch.stack(log_probs)
        
        advantage = returns - values.detach()
        policy_loss = -(log_probs * advantage).mean()
        value_loss = F.mse_loss(values, returns)
        loss = policy_loss + value_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Episode {ep+1}/{num_episodes}, Loss: {loss.item():.4f}, Total Reward: {np.sum(rewards):.4f}")
    
    return policy_net

trained_policy = train_agent(policy_net, optimizer, actions, combined_simulations, num_episodes=100)

# -----------------------------
# 4. Testing on Out-of-Sample Data (2016-02-01 to 2018-02-01)
# -----------------------------
env_test = PortfolioEnv(test_data, window_obs=60, window_state=120)

def test_agent(env, policy_net, actions):
    state = env.reset()
    done = False
    test_rewards = []
    while not done:
        obs, port_state = state
        with torch.no_grad():
            policy_logits, _ = policy_net(obs, port_state)
            probs = F.softmax(policy_logits, dim=1)
        action_idx = torch.argmax(probs, dim=1)
        action = actions[action_idx.item()]
        state, reward, done, _ = env.step(action)
        test_rewards.append(reward)
    total_return = np.prod([1 + r for r in test_rewards])
    return total_return

rl_test_return = test_agent(env_test, trained_policy, actions)
print("RL test return:", rl_test_return)

# -----------------------------
# 5. Benchmark Evaluation on Fixed Out-of-Sample Intervals
# -----------------------------
# Define evaluation intervals entirely within the test period.
intervals = [("2016-02-01", "2017-01-01"),
             ("2017-01-01", "2018-02-01")]

def sharpe_ratio(rewards, risk_free_rate=0.04, periods_per_year=252):
    rewards = np.array(rewards)
    mean_return = np.mean(rewards)
    std_return = np.std(rewards)
    if std_return == 0:
        return np.nan
    return mean_return / std_return * np.sqrt(periods_per_year)

def test_agent_with_rewards(env, policy_net, actions):
    state = env.reset()
    done = False
    rewards = []
    while not done:
        obs, port_state = state
        with torch.no_grad():
            policy_logits, _ = policy_net(obs, port_state)
            probs = F.softmax(policy_logits, dim=1)
        action_idx = torch.argmax(probs, dim=1)
        action = actions[action_idx.item()]
        state, reward, done, _ = env.step(action)
        rewards.append(reward)
    total_return = np.prod([1 + r for r in rewards])
    return total_return, rewards

def simulate_benchmark(env, weights):
    state = env.reset()
    done = False
    rewards = []
    while not done:
        state, reward, done, _ = env.step(weights)
        rewards.append(reward)
    total_return = np.prod([1 + r for r in rewards])
    return total_return, rewards

def optimize_portfolio(mu, cov):
    n = len(mu)
    w0 = np.ones(n) / n
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = [(0, 1) for _ in range(n)]
    def neg_sharpe(w, mu, cov):
        port_return = np.dot(w, mu)
        port_std = np.sqrt(np.dot(w, np.dot(cov, w)))
        return -port_return / port_std if port_std > 0 else 1e6
    result = minimize(neg_sharpe, w0, args=(mu, cov), bounds=bounds, constraints=constraints)
    return result.x if result.success else w0

results = {}

for start, end in intervals:
    interval_df = test_data.loc[start:end].copy()
    window_obs = 60
    window_state = 120
    if len(interval_df) < (window_obs + 1):
        print(f"Interval {start} to {end} is too short. Skipping.")
        continue
    env_interval = PortfolioEnv(interval_df, window_obs=window_obs, window_state=window_state)
    
    # RL Agent evaluation:
    rl_total_return, rl_rewards = test_agent_with_rewards(env_interval, trained_policy, actions)
    rl_sharpe = sharpe_ratio(rl_rewards)
    
    # MVO Benchmark:
    returns = np.log(interval_df / interval_df.shift(1)).dropna()
    mu_interval = returns.mean().values
    cov_interval = returns.cov().values
    mvo_weights = optimize_portfolio(mu_interval, cov_interval)
    mvo_total_return, mvo_rewards = simulate_benchmark(env_interval, mvo_weights)
    mvo_sharpe = sharpe_ratio(mvo_rewards)
    
    # Equal Weights Benchmark:
    n_assets = interval_df.shape[1]
    equal_weights = np.ones(n_assets) / n_assets
    equal_total_return, equal_rewards = simulate_benchmark(env_interval, equal_weights)
    equal_sharpe = sharpe_ratio(equal_rewards)
    
    results[(start, end)] = {
        "RL": {"Total Return": rl_total_return, "Sharpe": rl_sharpe},
        "MVO": {"Total Return": mvo_total_return, "Sharpe": mvo_sharpe},
        "Equal_weights": {"Total Return": equal_total_return, "Sharpe": equal_sharpe}
    }
    
    print(f"Interval {start} to {end}:")
    print(f"  RL            - Total Return: {rl_total_return:.4f}, Sharpe Ratio: {rl_sharpe:.4f}")
    print(f"  MVO           - Total Return: {mvo_total_return:.4f}, Sharpe Ratio: {mvo_sharpe:.4f}")
    print(f"  Equal Weights - Total Return: {equal_total_return:.4f}, Sharpe Ratio: {equal_sharpe:.4f}")
    print("---------------------------------------------------")
